# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# show all column 
pd.set_option('display.max_columns', 500)

In [23]:
def scraping_BA(pages, url = 'https://www.airlinequality.com/airline-reviews/british-airways'):
    base_url = url
    page = pages # 35
    page_size = 100

    users = []
    rates = []
    reviews = []
    detail = []

    # for i in range(1, pages + 1):
    for i in range(1, page + 1):

        print(f"Scraping page {i}")

        # Create URL to collect links from paginated data
        url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

        # Collect HTML data from this page
        response = requests.get(url)

        # Parse content
        content = response.content
        parsed_content = BeautifulSoup(content, 'html.parser')

        # get the users data
        user_review = parsed_content.find_all("h3", {"class": "text_sub_header userStatusWrapper"})
        for para in user_review:
            users.append(para.get_text())

        # get rating from user
        rate_review = parsed_content.find_all("div", {"class":"rating-10"})
        for para in rate_review[1:]:
            rates.append(para.get_text())

        # get text review data
        text_review = parsed_content.find_all("div", {"class": "text_content"})
        for para in text_review:
            reviews.append(para.get_text())

        # get all data inside table
        tabs = parsed_content.find_all("table", {"class": "review-ratings"})
        for para in tabs[1:]:
            datas = para.find_all("tr")
            rows = []
            for row in datas:
                one_row = row.find_all("td")
                for txts in one_row:
                    if "star fill" in str(txts):
                        star_fill = re.findall(r'star fill">(\d+)', str(txts))[-1]
                        rows.append(str(star_fill))
                    else:
                        rows.append(txts.get_text())
            detail.append(rows)

    print('finish')
    print(f"{len(users)} total users")
    print(f"{len(rates)} total rates")
    print(f"{len(detail)} total detail rates")
    print(f"{len(reviews)} total reviews")
    
    return pd.DataFrame({'users':users, 'detail':detail, 'rates':rates, 'reviews':reviews})

In [24]:
BA = scraping_BA(35)

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
finish
3471 total users
3471 total rates
3471 total detail rates
3471 total reviews


In [25]:
BA

,users,detail,rates,reviews
0,\n\nKeat Tan (United Kingdom) 7th February 2023,"[Aircraft, Boeing 787-9, Type Of Traveller, Co...",\n9/10\n,✅ Trip Verified | BA 242 on the 6/2/23. Boardi...
1,\n\nAdam Churchill (Spain) 6th February 2023,"[Aircraft, A320, Type Of Traveller, Solo Leisu...",\n10/10\n,✅ Trip Verified | Not only my first flight in...
2,\n\nMaria del Carmen Riesco Martin (United Kin...,"[Type Of Traveller, Family Leisure, Seat Type,...",\n10/10\n,✅ Trip Verified | My husband and myself were ...
3,\n\nS Anderson (United Kingdom) 31st January 2023,"[Aircraft, A320, Type Of Traveller, Business, ...",\n10/10\n,✅ Trip Verified | Organised boarding process. ...
4,\n\nM Wilson (United Kingdom) 31st January 2023,"[Type Of Traveller, Couple Leisure, Seat Type,...",\n2/10\n,✅ Trip Verified | Outward journey BA245 Londo...
...,...,...,...,...
3466,\n\nC Mcculloch (United Kingdom) 29th August 2012,"[Seat Type, Economy Class, Seat Comfort, 3, Ca...",\n7/10\n,Flight from Heathrow to Toronto. Booked emerge...
3467,\n\nNick Berry (United Kingdom) 28th August 2012,"[Seat Type, Business Class, Seat Comfort, 4, C...",\n9/10\n,LHR to HAM. Purser addresses all club passenge...
3468,\n\nAvril Barclay (United Kingdom) 12th Octobe...,"[Seat Type, Economy Class, Value For Money, 4,...",\n5/10\n,My son who had worked for British Airways urge...
3469,\n\nC Volz (United States) 11th October 2011,"[Seat Type, Premium Economy, Seat Comfort, 1, ...",\n4/10\n,London City-New York JFK via Shannon on A318 b...


In [26]:
BA.rates.unique()

array(['\n9/10\n', '\n10/10\n', '\n2/10\n', '\n4/10\n', '\n3/10\n',
       '\n1/10\n', '\n5/10\n', '\n8/10\n', '\n6/10\n', '\n7/10\n',
       '\r\n                        na\r\n                    '],
      dtype=object)

In [27]:
BA.to_csv('raw_data.csv', index=False)

In [28]:
def cleaning_data(df):
    # replace \n
    df = df.replace('\n','', regex=True)
    # split users data
    users = df['users'].str.extract(r'(.*)[(](.*)[)](.*)', expand=True)
    df.insert(0, 'Passengers', users[0])
    df.insert(1, 'Nationality', users[1])
    df.insert(2, 'Date Review', users[2])
    df.drop('users', axis=1, inplace=True)
    #clean rates column
    df['rates'] = df['rates'].str.split('/').str[0].str.strip()
    # get verified users data by split reviews columns
    df['verified'] = ['verified' if 'Trip Verified' in x else "not verified" for x in df['reviews']]
    # clean reviews column
    df['reviews'] = [x if '|' not in x else x.split('|')[1].strip() for x in df['reviews']]
    # create new dataframe from detail
    detail_df = pd.DataFrame(dict(zip(x[::2], x[1::2])) for x in df['detail'])
    df.drop('detail', axis=1, inplace=True)
    result = pd.concat([df,detail_df], axis=1)
    return result

In [48]:
df = cleaning_data(BA)

In [49]:
df[:2]

,Passengers,Nationality,Date Review,rates,reviews,verified,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value For Money,Recommended,Wifi & Connectivity
0,Keat Tan,United Kingdom,7th February 2023,9,BA 242 on the 6/2/23. Boarding was delayed due...,verified,Boeing 787-9,Couple Leisure,Business Class,Mexico City to London,February 2023,5,5,4,4,5,5,yes,NaN
1,Adam Churchill,Spain,6th February 2023,10,"Not only my first flight in 17 years, but also...",verified,A320,Solo Leisure,Economy Class,Palma to Gatwick,January 2023,5,5,5,NaN,5,5,yes,NaN


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3471 entries, 0 to 3470
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Passengers              3469 non-null   object
 1   Nationality             3469 non-null   object
 2   Date Review             3469 non-null   object
 3   rates                   3471 non-null   object
 4   reviews                 3471 non-null   object
 5   verified                3471 non-null   object
 6   Aircraft                1816 non-null   object
 7   Type Of Traveller       2701 non-null   object
 8   Seat Type               3469 non-null   object
 9   Route                   2697 non-null   object
 10  Date Flown              2693 non-null   object
 11  Seat Comfort            3371 non-null   object
 12  Cabin Staff Service     3363 non-null   object
 13  Food & Beverages        3134 non-null   object
 14  Inflight Entertainment  2439 non-null   object
 15  Grou

In [51]:
cols = df.columns.tolist()
for c in cols:
    if len(df[c].unique()) <= 20:
        print(f'{c} : {df[c].unique()}')

rates : ['9' '10' '2' '4' '3' '1' '5' '8' '6' '7' 'na']
verified : ['verified' 'not verified']
Type Of Traveller : ['Couple Leisure' 'Solo Leisure' 'Family Leisure' 'Business' nan]
Seat Type : ['Business Class' 'Economy Class' 'Premium Economy' 'First Class' nan]
Seat Comfort : ['5' '3' '4' '1' '2' nan]
Cabin Staff Service : ['5' '1' '4' '2' nan '3']
Food & Beverages : ['4' '5' '1' nan '2' '3' 'N/A']
Inflight Entertainment : ['4' nan '1' '3' '5' '2' 'N/A']
Ground Service : ['5' '4' '1' nan '3' '2']
Value For Money : ['5' '1' '3' '4' '2' 'N/A']
Recommended : ['yes' 'no']
Wifi & Connectivity : [nan '5' '1' '3' '4' '2']


In [52]:
df['rates'].replace('na', 0, inplace=True)
df['Food & Beverages'].replace('N/A', 0, inplace=True)
df['Inflight Entertainment'].replace('N/A', 0, inplace=True)
df['Value For Money'].replace('N/A', 0, inplace=True)

In [53]:
df['Date Review'] = pd.to_datetime(df['Date Review'])
df['Date Flown'] = pd.to_datetime(df['Date Flown'])
df['rates'] = pd.to_numeric(df['rates'])
df['Seat Comfort'] = pd.to_numeric(df['Seat Comfort'])
df['Cabin Staff Service'] = pd.to_numeric(df['Cabin Staff Service'])
df['Food & Beverages'] = pd.to_numeric(df['Food & Beverages'])
df['Inflight Entertainment'] = pd.to_numeric(df['Inflight Entertainment'])
df['Ground Service'] = pd.to_numeric(df['Ground Service'])
df['Value For Money'] = pd.to_numeric(df['Value For Money'])
df['Wifi & Connectivity'] = pd.to_numeric(df['Wifi & Connectivity'])

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3471 entries, 0 to 3470
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Passengers              3469 non-null   object        
 1   Nationality             3469 non-null   object        
 2   Date Review             3469 non-null   datetime64[ns]
 3   rates                   3471 non-null   object        
 4   reviews                 3471 non-null   object        
 5   verified                3471 non-null   object        
 6   Aircraft                1816 non-null   object        
 7   Type Of Traveller       2701 non-null   object        
 8   Seat Type               3469 non-null   object        
 9   Route                   2697 non-null   object        
 10  Date Flown              2693 non-null   datetime64[ns]
 11  Seat Comfort            3371 non-null   float64       
 12  Cabin Staff Service     3363 non-null   float64 

In [55]:
df.to_csv('data/BA_reviews.csv', index=False)